In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
#
# Copyright (C) 2024 Bitergia

import sys
sys.path.append('.')

import config as cfg
import pandas as pd
import plotly.express as px

from datetime import datetime
from dateutil.relativedelta import relativedelta
from lib.grimoirelab_queries_prototype import GrimoireLabQuery


In [2]:
user = cfg.credentials['username']
password = cfg.credentials['password']
host = cfg.host
port = cfg.port
path = cfg.path
use_ssl = cfg.use_ssl
verify_certs = cfg.verify_certs
timeout = cfg.timeout
max_retries = cfg.max_retries

In [3]:
glabquery = GrimoireLabQuery(user, password, host, port, path, use_ssl, verify_certs,timeout, max_retries)   
glabquery.set_preffix_index('intersectmbo')



In [4]:
# Time is set to past month

now = datetime.now()

start_of_current_month = now.replace(day=1, hour=0, minute=0, second=0, microsecond=0)
start_of_previous_month = start_of_current_month + relativedelta(months=-1)

start_date_str = start_of_previous_month.strftime("%Y-%m-%d")
end_date_str = start_of_current_month.strftime("%Y-%m-%d")

In [5]:
# Project to be excluded

excluded_projects = ["Marlowe","Daedalus","Hydra","Cardano Wallet","Aiken","Pallas","Oura","Partner Chains","Cardano Improvement Proposals (CIPS)","Cardano Crypto","Catalyst","Cardano Base","Mithril","Cardano Automation","Lace","Wallet / Adrestria","Amaru"]

In [6]:
glabquery.date_range(start_date_str, end_date_str)
glabquery.filter_out_projects(excluded_projects)


# Filters

Before getting the results, be aware of the filters we are using.

 - Bot activity is always excluded
 - In commits, merge commits (the ones that modify 0 files) are excluded.
 - The list of projects above is also excluded
 - The date range is:

In [7]:
print("Data range: From %s to %s" % (start_date_str, end_date_str))

Data range: From 2024-11-01 to 2024-12-01


# GitHub Overview

In [8]:
results = {"# Commits": glabquery.metric_commits(),
           "# Authors": glabquery.metric_commit_authors(),
           "# Repositories": glabquery.metric_commit_repositories()}
df = pd.DataFrame([results])
df.head(5)

,# Commits,# Authors,# Repositories
0,1269,87,24


# GitHub Overview Organizations

In [9]:
df = glabquery.df_commit_activity_per_org()
df.head(15)

,org_name,unique_authors,total_files,total_lines_added,total_lines_removed,total_lines_changed,unique_repos,unique_projects,unique_commits
0,Input Output (IOHK),46,6521.0,330687.0,102424.0,433111.0,12,8,752
1,Well-Typed,5,1356.0,34600.0,29614.0,64214.0,3,3,191
2,Intersect MBO,16,361.0,3395.0,966.0,4361.0,9,3,87
3,Tweag,3,1136.0,21402.0,19853.0,41255.0,5,3,78
4,BinarApps,2,3906.0,33459.0,49634.0,83093.0,2,2,61
5,Unknown,8,124.0,3086.0,1457.0,4543.0,6,5,35
6,Sireto Technology,1,2621.0,9896.0,287.0,10183.0,2,2,29
7,Quviq,2,56.0,1467.0,1977.0,3444.0,2,1,20
8,Arcor lhr,1,21.0,594.0,651.0,1245.0,1,1,10
9,Associe,1,4.0,116.0,0.0,116.0,1,1,4


# Commits by timezone

In [10]:
df = glabquery.df_commit_per_timezone()
df.head(10)

,timezone,unique_commits
0,1,686
1,0,296
2,-6,67
3,-7,60
4,-4,45
5,2,45
6,-5,34
7,5,28
8,-8,5
9,-3,1


# GitHub Overview - Repository Activity

In [11]:
df = glabquery.df_commit_activity_per_repository()
df.head(15)

,github_repo,unique_authors,total_files,total_lines_added,total_lines_removed,total_lines_changed,unique_orgs,unique_projects,unique_commits,avg_lines_changed,avg_files
0,intersectmbo/lsm-tree,7,621.0,14566.0,8739.0,23305.0,2,1,153,152.320261,4.058824
1,intersectmbo/ouroboros-consensus,13,1889.0,17918.0,18768.0,36686.0,3,1,124,295.854839,15.233871
2,intersectmbo/cardano-node-tests,2,432.0,5557.0,5294.0,10851.0,1,1,107,101.411215,4.037383
3,intersectmbo/ouroboros-network,6,1028.0,28170.0,31910.0,60080.0,2,1,106,566.792453,9.698113
4,intersectmbo/cardano-cli,7,1303.0,23570.0,20936.0,44506.0,2,1,105,423.866667,12.409524
5,intersectmbo/plutus,12,1976.0,211458.0,14236.0,225694.0,3,1,103,2191.203883,19.184466
6,intersectmbo/cardano-api,7,224.0,3635.0,1504.0,5139.0,2,1,102,50.382353,2.196078
7,intersectmbo/cardano-node,14,286.0,64880.0,33177.0,98057.0,3,1,102,961.343137,2.803922
8,intersectmbo/cardano-ledger,8,1073.0,12980.0,14674.0,27654.0,3,1,97,285.092784,11.061856
9,intersectmbo/govtool,6,324.0,21570.0,49299.0,70869.0,4,1,80,885.862500,4.050000


# GitHub Issues by Repo

In [12]:
df = glabquery.df_issues_per_repository()
df.head(15)

,github_repo,issues,unique_authors,avg_time_open_days
0,intersectmbo/govtool,80,8,8.370500
1,intersectmbo/plutus,24,9,5.655833
2,intersectmbo/ouroboros-consensus,16,3,4.122500
3,intersectmbo/cardano-ledger,13,6,5.018461
4,intersectmbo/cardano-db-sync,11,6,3.647273
5,intersectmbo/cardano-api,5,4,7.300000
6,intersectmbo/cardano-cli,4,2,10.427500
7,intersectmbo/formal-ledger-specifications,4,2,15.455000
8,intersectmbo/cardano-node,3,3,7.076667
